In [1]:
# 폰트 다운로드 (런타임 재부팅 해야함)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 2s (4,164 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 122349 files and di

In [2]:
# plot 라이브러리
import matplotlib
import matplotlib.pyplot as plt
# 경고 메시지 무시
import warnings
# 실행결과 경고메시지 출력 제외
warnings.filterwarnings('ignore')
# 글꼴 설정
plt.rc('font', family='NanumBarunGothic') 


ALL => 개 개  고양이 고양이 고양이 고양이 고양이 고양이 고양이

* 예를들면 극단적으로 데이터가 개는 앞쪽에 쏠리고 고양이는 뒷쪽에 쏠릴경우
  학습 할 수록 개를 학습한 내용이 희미해지고 고양이에 쏠리게됨
  

In [4]:
# 데이터 split하기 위한 도구
from sklearn.model_selection import train_test_split

# 데이터 고차원 화 
from sklearn.preprocessing import PolynomialFeatures

# 데이터 rescaling
from sklearn.preprocessing import StandardScaler

# 사이킷런 전처리 파이프라인 만들기 위해
from sklearn.pipeline import Pipeline

# 행렬 연산 라이브러리
import numpy as np

# 사이킷런의 linear model 중 stochastic gradient descent 
from sklearn.linear_model import SGDRegressor

# 사이킷런의 loss function 중 하나인 mean squared error 모듈 (MSE)
from sklearn.metrics import mean_squared_error

# 시스템 접근 모듈
import os

# random 기준
np.random.seed(42)

# 그림을 저장할 폴더
PROJECT_ROOT_DIR = '.'

# 100개 샘플 예정
m=100

# 100개의 샘플 사용 데이터
X = 6 * np.random.rand(m, 1)-3

# 100개의 샘플 사용 데이터의 정답값
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

# train set, validation set                     # ravel 다차원을 1차원으로,
                                                # test_size = validation set 50%, random_state suffling 기준
X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state =10)

print(len(X_train))
print(len(X_val))
print(len(y_train))
print(len(y_val))

25
25
25


In [ ]:
# 전처리 파이프라인 polynomial Features의 차수는 2
poly_scaler = Pipeline([
    ("poly_features", PolynomialFeatures(degree = 2, include_bias=False)),
    ("std_scaler", StandardScaler()),
])

# X로 fitting 한 다음에 변환까지
X_train_poly_scaled = poly_scaler.fit_transform(X_train)

# 이미 X_train으로 poly_scaler 를 fitting 해서 변환만 해줌 / 데이터가 상대적으로 많은 X_train으로 fitting 시킴
X_val_poly_scaled = poly_scaler.transform(X_val)


# SGDRegressor 모델 정의
# penalty: {'l2', 'l1', 'elasticnet'}, 기본값='l2' 사용할 페널티(정규화 용어라고도 함)입니다. 선형 SVM 모델의 표준 정규화 장치인 'l2'가 기본값입니다. 'l1' 및 'elasticnet'은 'l2'로 달성할 수 없는 모델(기능 선택)에 희소성을 가져올 수 있습니다.
# eta0 float, 기본값=0.01 초기 학습률입니다. 기본값은 0.01입니다.
# warm_start의 default는 False임. 이는 .fit을 실행할 때, 이전에 업데이트된 weight(coefficient)를 초기화하고 다시 fitting한다는 것을 의미한다. 반대로 True는 이전 호출에 사용했던 solution을 재사용 할지 여부 결정
# learning_rate="constant" : Learning Rate로 지정한 상수값을 계속 사용하는 것을 나타냅니다.
sgd_reg = SGDRegressor(max_iter=1,                 # 횟수 설정, 여기에 500을 넣지 않는 이유는 max_iter가 없는 모델들도 있어서 일단 체험
                       penalty=None,               #
                       eta0=0.0005,                # learning rate 수치
                       warm_start=True,            # ★
                       learning_rate="constant",   # 
                       random_state=42)            #

# 학습 횟수
n_epochs = 500

# 학습 오차와 validation 오차를 담을 리스트 그릇 선언
train_errors, val_errors = [], []

# 학습 반복 (500번)
for epoch in range(n_epochs):
  # fitting, polynomial 시킨거랑 정답값으로 fitting,
  sgd_reg.fit(X_train_poly_scaled, y_train)
  
  # fitting된 모델에 train X와 validation X 넣어서 각각 예측값 도출                  # 반복문 루프마다 fitting 시키고, 곧 바로 predict를 시키고                          //  over fitting
  y_train_predict = sgd_reg.predict(X_train_poly_scaled)                             # MSE를 만들고, 리스트에 넣어서 몇번째 반복학습할때 에러가 제일 적은지 확인
  y_val_predict = sgd_reg.predict(X_val_poly_scaled)

  # MSE 구한 후 리스트에 각각 담기
  train_errors.append(mean_squared_error(y_train, y_train_predict))
  val_errors.append(mean_squared_error(y_val, y_val_predict))

# argmin에서 몇번째에 val error가 낮냐? 즉 몇번째가 몇번째 epoch냐
best_epoch = np.argmin(val_errors)

# best epoch에서 val_error를 찾고, 그 값은 MSE라서 RMSE로 구함.
best_val_rmse = np.sqrt(val_errors[best_epoch])



In [ ]:
from sklearn.base import clone

sgd_reg = SGDRegressor(max_iter=1, warm_start=True, penalty=None,
                       learning_rate="constant", eta0=0.0005, random_state=42)


minimum_val_error = float("inf")

best_epoch = None
best_model = None

for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train)  # 이어서 학습합니다
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    if val_error < minimum_val_error:
        count=0
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)
    else: # 만약에 minimum error 가 연속으로 뜬다면 break 시킬거임
      count += 1
      if count==5:
        break         # early stopping (조기종료)

In [ ]:
# 전처리 파이프라인 polynomial Features의 차수는 90
poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler()),
    ])

# X로 fitting 한다음에 변환까지
X_train_poly_scaled = poly_scaler.fit_transform(X_train)

# 이미 X_train으로 poly_scaler를 fitting해서 변환만 해줌.
X_val_poly_scaled = poly_scaler.transform(X_val)

# SGDRegressor 모델 정의


# penalty: {'l2', 'l1', 'elasticnet'}, 기본값='l2' 사용할 페널티(정규화 용어라고도 함)입니다. 선형 SVM 모델의 표준 정규화 장치인 'l2'가 기본값입니다. 'l1' 및 'elasticnet'은 'l2'로 달성할 수 없는 모델(기능 선택)에 희소성을 가져올 수 있습니다.
# eta0 float, 기본값=0.01 초기 학습률입니다. 기본값은 0.01입니다.
# warm_start의 default는 False임. 이는 .fit을 실행할 때, 이전에 업데이트된 weight(coefficient)를 초기화하고 다시 fitting한다는 것을 의미한다. 반대로 True는 이전 호출에 사용했던 solution을 재사용 할지 여부 결정
# learning_rate="constant" : Learning Rate로 지정한 상수값을 계속 사용하는 것을 나타냅니다.
sgd_reg = SGDRegressor(max_iter=1,
                       penalty=None,
                       eta0=0.0005,
                       warm_start=True,
                       learning_rate="constant",
                       random_state=42)

# 학습 횟수
n_epochs = 500

# 학습오차와 validation 오차 담을 리스트 그릇
train_errors, val_errors = [], []

# 학습 반복
for epoch in range(n_epochs):
    # fitting, polynomial 시킨거랑 정답값으로 fitting.
    sgd_reg.fit(X_train_poly_scaled, y_train)

    # fitting된 모델에 train X와 validation X 넣어서 각각 예측값 도출
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)

    # MSE 구한 후 리스트에 각각 담기
    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(mean_squared_error(y_val, y_val_predict))

# argmin에서 몇번째에 val error가 낮냐? 즉 몇번째가 몇번째 epoch냐
best_epoch = np.argmin(val_errors)

# best epoch에서 val_error를 찾고, 그 값은 MSE라서 RMSE로 구함.
best_val_rmse = np.sqrt(val_errors[best_epoch])

# plot에서 설정할 값들.
# xytext : 텍스트 위치 
# xy : 화살표 위치
# ha : horizontal alignment
# arrowprops : 화살표 속성들

plt.annotate('최선의 모델',
             xy=(best_epoch, best_val_rmse),
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,
            )

best_val_rmse -= 0.03  # 그래프를 더 보기 좋게 만들기 위해
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)

# val_errors와 train_errors의 RMSE
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="검증 세트")
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="훈련 세트")

# 각 화살표 네이밍
plt.legend(loc="upper right", fontsize=14)

# xlabel
plt.xlabel("에포크", fontsize=14)

# ylabel
plt.ylabel("RMSE", fontsize=14)
save_fig("early_stopping_plot")
plt.show()

